# fusionのシミュレーション結果をvtuで出力して、それをstlで出力するためのコードす。

コードを実行して3種類のvtuをアップロードしてください。



In [5]:
# ==========================================
# 1. ライブラリのインストール
# ==========================================
!pip -q install pyvista vtk

In [6]:
# ==========================================
# 自動合体＆変形・STL変換ツール（3つのVTU必須版）
#  - X, Y, Z 成分ごとのVTUファイル「3つ」が揃わないと処理しません
#  - 変位スケール調整可能（DISP_SCALE）
# ==========================================

!pip -q install pyvista vtk

from google.colab import files
import pyvista as pv
import numpy as np

# ============================
# ★ユーザー設定（ここを調整）
# ============================
DISP_SCALE = 3.0                 # 変位倍率
OUTPUT_NAME = "deformed_geometry.stl"

# ============================================================
print("▼ ファイルをアップロードしてください")
print("・変位 X, Y, Z 成分ごとのVTUファイル（必ず3つ）")
uploaded = files.upload()

# ============================
# 3つ以外は即エラー
# ============================
if len(uploaded.keys()) != 3:
    raise RuntimeError(
        f"❌ エラー: アップロードされたファイル数が {len(uploaded.keys())} 個です。\n"
        "このツールは X, Y, Z 成分のVTUを「ちょうど3つ」アップロードしてください。"
    )

# データを格納する変数の初期化
mesh_base = None
disp_x = None
disp_y = None
disp_z = None
x_name = y_name = z_name = None

print(f"\n{'='*50}")
print("解析と統合処理を開始します（3ファイル必須モード）")
print(f"{'='*50}")

def is_scalar_array(arr):
    arr = np.asarray(arr)
    return (arr.ndim == 1) or (arr.ndim == 2 and arr.shape[1] == 1)

def pick_component_from_mesh(mesh):
    """
    そのVTUが X/Y/Z のどれを含んでいるか判定して返す
    戻り値: ("x" or "y" or "z" or None, array_name or None, values or None)
    """
    for name in mesh.array_names:
        lower_name = name.lower()
        is_disp = ("displacement" in lower_name) or ("変位" in name)
        if not is_disp:
            continue

        arr = mesh.get_array(name)
        if arr is None or not is_scalar_array(arr):
            continue

        vals = np.asarray(arr).reshape(-1)

        # 成分判定（できるだけ確実寄り）
        if (":x" in lower_name) or ("_x" in lower_name) or lower_name.endswith("x"):
            return "x", name, vals
        if (":y" in lower_name) or ("_y" in lower_name) or lower_name.endswith("y"):
            return "y", name, vals
        if (":z" in lower_name) or ("_z" in lower_name) or lower_name.endswith("z"):
            return "z", name, vals

    return None, None, None


try:
    # アップロードされた3ファイルを読む
    for filename in uploaded.keys():
        print(f"読み込み中: {filename} ...")
        mesh = pv.read(filename)

        # ベースメッシュ確保
        if mesh_base is None:
            mesh_base = mesh

        # 点数一致確認
        if mesh.n_points != mesh_base.n_points:
            raise RuntimeError(
                f"❌ エラー: {filename} の点数({mesh.n_points})が他と一致しません({mesh_base.n_points})。"
                "同一メッシュ由来のX/Y/Zファイルをアップしてください。"
            )

        comp, arr_name, vals = pick_component_from_mesh(mesh)
        if comp is None:
            raise RuntimeError(
                f"❌ エラー: {filename} から変位成分（X/Y/Z）を特定できませんでした。\n"
                "配列名に '変位' または 'Displacement' が含まれ、かつ ':X'/'_X' 等で成分が判別できる必要があります。"
            )

        if comp == "x":
            if disp_x is not None:
                raise RuntimeError("❌ エラー: X成分ファイルが重複しています（Xが2つ以上検出）。")
            disp_x, x_name = vals, arr_name
            print(f"  + X成分を採用: '{arr_name}'")
        elif comp == "y":
            if disp_y is not None:
                raise RuntimeError("❌ エラー: Y成分ファイルが重複しています（Yが2つ以上検出）。")
            disp_y, y_name = vals, arr_name
            print(f"  + Y成分を採用: '{arr_name}'")
        elif comp == "z":
            if disp_z is not None:
                raise RuntimeError("❌ エラー: Z成分ファイルが重複しています（Zが2つ以上検出）。")
            disp_z, z_name = vals, arr_name
            print(f"  + Z成分を採用: '{arr_name}'")

    # 3成分が必ず揃っていることを保証
    if disp_x is None or disp_y is None or disp_z is None:
        raise RuntimeError(
            "❌ エラー: X/Y/Z のいずれかが見つかりませんでした。\n"
            f"検出状況: X={'OK' if disp_x is not None else 'NG'}, "
            f"Y={'OK' if disp_y is not None else 'NG'}, "
            f"Z={'OK' if disp_z is not None else 'NG'}\n"
            "X/Y/Z がそれぞれ1つずつ含まれるVTUを3つアップしてください。"
        )

    print("\nX, Y, Z の3成分が揃いました。データを結合して変形します...")
    print(f"  - 変位倍率 (DISP_SCALE) = {DISP_SCALE}")
    print(f"  - 採用配列名: X='{x_name}', Y='{y_name}', Z='{z_name}'")

    # ベクトルを作成して（ここで倍率を適用）メッシュに追加
    vectors = np.column_stack((disp_x, disp_y, disp_z)) * DISP_SCALE
    combined_name = "Combined_Displacement"
    mesh_base.point_data[combined_name] = vectors

    # 変形
    final_mesh = mesh_base.warp_by_vector(combined_name, factor=1.0)

    # 表面抽出（STL化のため）
    surface = final_mesh.extract_surface().triangulate().clean()

    # 保存
    surface.save(OUTPUT_NAME)

    print(f"\n✅ 変換成功！: {OUTPUT_NAME}")
    files.download(OUTPUT_NAME)

except Exception as e:
    print(f"\n予期せぬエラーが発生しました: {e}")
    raise


▼ ファイルをアップロードしてください
・変位 X, Y, Z 成分ごとのVTUファイル（必ず3つ）


Saving sim_x.vtu to sim_x (3).vtu
Saving sim_y.vtu to sim_y (3).vtu
Saving sim_z.vtu to sim_z (3).vtu

解析と統合処理を開始します（3ファイル必須モード）
読み込み中: sim_x (3).vtu ...
  + X成分を採用: '変位:X'
読み込み中: sim_y (3).vtu ...
  + Y成分を採用: '変位:Y'
読み込み中: sim_z (3).vtu ...
  + Z成分を採用: '変位:Z'

X, Y, Z の3成分が揃いました。データを結合して変形します...
  - 変位倍率 (DISP_SCALE) = 3.0
  - 採用配列名: X='変位:X', Y='変位:Y', Z='変位:Z'

✅ 変換成功！: deformed_geometry.stl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>